<a href="https://colab.research.google.com/github/NS-y0/GovTools/blob/master/banjectionTracker/y0%E2%80%99s_Banjections_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **About** 
---
**Created on**: 2020 July 22

**Author**: y0

**Version**: 0.1

**Contact**: 
*   Discord DM: y0#5738
*   NS Telegram: Evrigenis

**Modified from**: [Valentine-Z's Bank Retriever](https://colab.research.google.com/drive/1Rrd20eBdX-MvLkUHJGHhIdJP3SrjPbtE). As per [this discord message](https://discord.com/channels/539090576738353152/539217730881191957/681570395844902962), the Bank Retriever is in the public domain.

**Purpose**: The aim of this code is for NationStates users to enter their region (or anyone else's, if they want) and see who was banjected in the last week.
This code only extracts the information through public means (the NS API).
IT DOES NOT ASK FOR YOUR PASSWORD, OR ANY OTHER SENSITIVE INFORMATION.
All you need is a Google account, which you can use a burner account for privacy.

**How to use**: Change the parameters, go to "Runtime" and press "Run all."

**Shareable Link**: https://colab.research.google.com/drive/1mUtnmdpjvsexHMmVYIR5PhM0BVz5pDiG?usp=sharing

**License**:

Copyright © 2020 y0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

[www.apache.org/licenses/LICENSE-2.0](https://)



Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# The Program

##Parameters
You must enter these values!

In [ ]:
# USER AGENT - IMPORTANT! PLEASE CHANGE TO YOUR MAIN NATION FOR VALIDATION PURPOSE.
main_nation = "YOUR NATION HERE" #@param{type:"string"}

# This is where your list of regions go. LEAVE THE QUOTES ON EITHER END.
# It must either be a list of names with \n (Enter) for each line
# or be comma-separated, e.g. "region1, region2".
# It is NOT case sensitive. It may use spaces or underscores.
region_list = "the_east_pacific, the north pacific, some other region" #@param{type:"string"}

But these are optional

In [ ]:
# If you specify since_date, it must be in the format you have indicated.
# to change to mm/dd/yy, just switch the format to "%m/%d/%Y"
# See https://docs.python.org/3/library/time.html#time.strftime for more info
since_date = "2020-06-21" #@param {type:"date"}
date_format = "%Y-%m-%d" #@param{type:"string"}

# In case the API request rate goes too fast, modify this number. Some browsers run faster than others.
# ALWAYS make sure that the API request rate is within acceptable rate!
# In seconds. E.g. 0.400 seconds, 1 second, etc...
custom_delay = 0 #@param {type:"number"}

# The NS API occasionally includes information about nations which were ejected
# in regions that are NOT requested
# Setting this to True filters them out, and setting this to false alows them.
strict_filter = True #@param {type:"boolean"}

In [ ]:
# if you want to use the data elsewhere
export_file_name = "banjections" #@param {type:"string"}

# if set to True, your browser downloads the data
export_locally = False #@param {type:"boolean"}

## Retrieval

In [ ]:
import time
from time import sleep
from calendar import timegm
import calendar

import requests
import xml.etree.ElementTree as ElementTree
import re



#---------------------------------------#
# A little bit of pre-processing
headers = {'User-Agent': "y0's Ejection tracker. Built and maintained by https://nationstates.net/evrigenis, in use by " + main_nation}

region_array = re.split(',|\n', region_list)
list_of_regions = filter(None, region_array)

if time.strptime(time.strftime(date_format, time.gmtime()), date_format).tm_year == 1900:
  print("The date format is invalid. Proceeding with yyyy-mm-dd format")
  date_format = "%Y-%m-%d"

formatted_time = "0"
if since_date != "":
  try:
    formatted_time = str(calendar.timegm(time.strptime(since_date, date_format)))
  except:
    print("The date was entered incorrectly. Proceeding without entered date.")
#---------------------------------------#



#---------------------------------------#
# Variable Definitions
num_nations = 0
num_regions = 0

time_list = []
result_region_list = []
RO_list = []
ejected_list = []
was_banned_list = []
#---------------------------------------#



#---------------------------------------#
start_time = time.time()

for region in list_of_regions:
  try:
    request_details = "q=happenings;view=region." + region.replace(" ", "_") + ";filter=eject;sincetime=" + formatted_time
    url = "https://www.nationstates.net/cgi-bin/api.cgi?" + request_details
    r = requests.get(url, headers = headers)
    root = ElementTree.fromstring(r.content)

    print(region + ":")
    num_regions += 1

    events = root.find("HAPPENINGS").findall('EVENT')

    if events == []:
      raise AttributeError

    


    for scale in events:
      parse_me = scale.find('TEXT').text
 
      region_in_request = re.findall('(?<=%%)\w+', parse_me)[0]
      if (strict_filter and region_in_request != region):
        continue
      
      nations_in_request = re.findall('(?<=@@)\w+', parse_me)
      RO = nations_in_request[1]
      ejected_nation = nations_in_request[0]
      was_banned = re.search(r'and banned', parse_me) != None

      nations_in_request = re.findall('(?<=@@)\w+', parse_me)

      timestamp = int(scale.find('TIMESTAMP').text)
      human_time = time.strftime(date_format, time.gmtime(timestamp))
      
      print('%15s' % "Date " + human_time + ",",
            '%5s' % "RO", RO + ",",
            '%25s' % "Ejected Nation", ejected_nation + ",",
            '%25s' % "Was banned", was_banned)

      result_region_list.append(region)
      time_list.append(human_time)
      RO_list.append(RO)
      ejected_list.append(ejected_nation)
      was_banned_list.append(was_banned)

      num_nations += 1
      sleep(custom_delay)
    print("\n")
                     

  except KeyboardInterrupt: # Interrupt with KB.
    print("\nINTERRUPTED! The error codes below are normal when interruption is executed.")
    duration = time.time() - start_time
    
    print("\n\n--- Total time of %s seconds ---" % (duration))
    print("INTERRUPTED Number of nations:",num_nations)
    print("INTERRUPTED Rate:",(num_nations / duration))
    print("Legal Rate:",50/30,"- 50 requests per 30 seconds.")
    raise

  except (AttributeError):
    print('%50s' % region,": Region may or may not exist, but it has no info.")
    
  finally:
    sleep(custom_delay)
#---------------------------------------#



# Crucial stats to make sure that the tool follows the API rate limit. NOTE THAT THE ACTUAL RETRIEVAL RATE MIGHT BE HIGHER OR LOWER, DEPENDING ON THE ALGORITHMS OR YOUR COMPUTER PERFORMANCE.
duration = time.time() - start_time
print("--- Total time of %s seconds ---" % (duration))
print("Number of ejected nations:",num_nations)
print("Number of requested regions:", num_regions)
print("Rate:",(num_regions / duration))
print("Legal Rate:",50/30,"- 50 requests per 30 seconds.")

##Export

In [ ]:
# To export the data as a copiable table,
# Click the table, press Ctrl + A to select the it all, and paste it where you wish.

import pandas as pd

pd.set_option('display.max_rows', None)

df1 = pd.DataFrame({"Region": result_region_list,
                    "Date/Time": time_list,
                    "RO": RO_list,
                    "Ejected Nation": ejected_list,
                    "Was banned": was_banned_list})

df1

In [ ]:
if (export_locally):
  from google.colab import files
  with open(export_file_name + '.csv', 'w') as f:
    df1.to_csv(f)
  files.download(export_file_name + '.csv')